# Prediction Reasoning Analysis

This notebook provides detailed reasoning for ML predictions stored in `eda_smartlist.us_wealth_management_smartlist.ML_predictions_single_policy`.

**Features:**
- Why each product was recommended (top contributing features)
- Key patterns influencing high confidence scores
- Why other products were not predicted (comparative analysis)
- Feature-level explanations using SHAP values


In [ ]:
# ============================================================================
# IMPORTS AND CONFIGURATION
# ============================================================================
import os
import pickle
from typing import Dict, Any, List, Tuple

import mlflow
import numpy as np
import pandas as pd
import shap
from mlflow.tracking import MlflowClient
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

# Configuration
PREDICTIONS_TABLE = "eda_smartlist.us_wealth_management_smartlist.ML_predictions_single_policy"
SOURCE_TABLE = "dl_tenants_daas.us_wealth_management.wealth_management_client_metrics"
MODEL_NAME = "eda_smartlist.models.lgbm_model_hyperparameter_last2products"
MODEL_VERSION = "1"
ARTIFACTS_LOCAL_PATH = "/Workspace/EDA_workspace/RA Smart List/Smart List (NEW)/multiproduct_smartlist/Final_model_files/artifacts.pkl"

# Initialize Spark
spark = SparkSession.builder.getOrCreate()

print("=" * 80)
print("PREDICTION REASONING ANALYSIS")
print("=" * 80)


In [ ]:
# ============================================================================
# PREPROCESSING HELPER FUNCTIONS (from training notebook)
# ============================================================================

def create_product_category_column(df):
    """Create product_category column from prod_lob, sub_product_level_1, and sub_product_level_2."""
    return df.withColumn(
        "product_category",
        F.when(F.col("prod_lob") == "LIFE", "LIFE_INSURANCE")
        .when(
            F.col("sub_product_level_1").isin(
                "VLI", "WL", "UL/IUL", "TERM", "PROTECTIVE PRODUCT"
            ),
            "LIFE_INSURANCE",
        )
        .when(F.col("sub_product_level_2").like("%LIFE%"), "LIFE_INSURANCE")
        .when(
            F.col("sub_product_level_2").isin(
                "VARIABLE UNIVERSAL LIFE",
                "WHOLE LIFE",
                "UNIVERSAL LIFE",
                "INDEX UNIVERSAL LIFE",
                "TERM PRODUCT",
                "VARIABLE LIFE",
                "SURVIVORSHIP WHOLE LIFE",
                "MONY PROTECTIVE PRODUCT",
            ),
            "LIFE_INSURANCE",
        )
        .when(
            F.col("prod_lob").isin("GROUP RETIREMENT", "INDIVIDUAL RETIREMENT"),
            "RETIREMENT",
        )
        .when(
            F.col("sub_product_level_1").isin(
                "EQUIVEST",
                "RETIREMENT 401K",
                "ACCUMULATOR",
                "RETIREMENT CORNERSTONE",
                "SCS",
                "INVESTMENT EDGE",
            ),
            "RETIREMENT",
        )
        .when(
            (F.col("sub_product_level_2").like("%403B%"))
            | (F.col("sub_product_level_2").like("%401%"))
            | (F.col("sub_product_level_2").like("%IRA%"))
            | (F.col("sub_product_level_2").like("%SEP%")),
            "RETIREMENT",
        )
        .when(F.col("prod_lob") == "BROKER DEALER", "INVESTMENT")
        .when(
            F.col("sub_product_level_1").isin(
                "INVESTMENT PRODUCT - DIRECT",
                "INVESTMENT PRODUCT - BROKERAGE",
                "INVESTMENT PRODUCT - ADVISORY",
                "DIRECT",
                "BROKERAGE",
                "ADVISORY",
                "CASH SOLICITOR",
            ),
            "INVESTMENT",
        )
        .when(
            (F.col("sub_product_level_2").like("%Investment%"))
            | (F.col("sub_product_level_2").like("%Brokerage%"))
            | (F.col("sub_product_level_2").like("%Advisory%")),
            "INVESTMENT",
        )
        .when(F.col("prod_lob") == "NETWORK", "NETWORK_PRODUCTS")
        .when(
            (F.col("sub_product_level_1") == "NETWORK PRODUCTS")
            | (F.col("sub_product_level_2") == "NETWORK PRODUCTS"),
            "NETWORK_PRODUCTS",
        )
        .when(
            (F.col("prod_lob") == "OTHERS") & (F.col("sub_product_level_1") == "HAS"),
            "DISABILITY",
        )
        .when(F.col("sub_product_level_2") == "HAS - DISABILITY", "DISABILITY")
        .when(F.col("prod_lob") == "OTHERS", "HEALTH")
        .when(F.col("sub_product_level_2") == "GROUP HEALTH PRODUCTS", "HEALTH")
        .otherwise("OTHER"),
    )


def add_asset_allocation_ratios(df):
    """Add asset allocation ratio features based on wealth metrics."""
    df = df.withColumn(
        "stock_allocation_ratio",
        F.col("wc_assetmix_stocks")
        / F.when(F.col("wc_total_assets") != 0, F.col("wc_total_assets")).otherwise(
            F.lit(None)
        ),
    )
    df = df.withColumn(
        "bond_allocation_ratio",
        F.col("wc_assetmix_bonds")
        / F.when(F.col("wc_total_assets") != 0, F.col("wc_total_assets")).otherwise(
            F.lit(None)
        ),
    )
    df = df.withColumn(
        "annuity_allocation_ratio",
        F.col("wc_assetmix_annuity")
        / F.when(F.col("wc_total_assets") != 0, F.col("wc_total_assets")).otherwise(
            F.lit(None)
        ),
    )
    df = df.withColumn(
        "mutual_fund_allocation_ratio",
        F.col("wc_assetmix_mutual_funds")
        / F.when(F.col("wc_total_assets") != 0, F.col("wc_total_assets")).otherwise(
            F.lit(None)
        ),
    )
    return df


def impute_missing_values(df, categorical_cols, categorical_mode="UNKNOWN"):
    """Impute missing values in DataFrame."""
    numeric_cols = [
        "first_acct_val_amt",
        "first_face_amt",
        "first_cash_val_amt",
        "wc_total_assets",
        "wc_assetmix_stocks",
        "wc_assetmix_bonds",
        "wc_assetmix_mutual_funds",
        "wc_assetmix_annuity",
        "wc_assetmix_deposits",
        "wc_assetmix_other_assets",
        "psn_age",
        "stock_allocation_ratio",
        "bond_allocation_ratio",
        "annuity_allocation_ratio",
        "mutual_fund_allocation_ratio",
        "age_at_first_policy",
        "years_to_second_policy",
    ]

    # Median imputation for numeric columns
    fill_numeric = {}
    for c in numeric_cols:
        if c in df.columns:
            try:
                median_val = df.approxQuantile(c, [0.5], 0.001)[0]
            except Exception:
                median_val = 0.0
            if median_val is None:
                median_val = 0.0
            fill_numeric[c] = float(median_val)

    result_df = df.fillna(fill_numeric)

    # Mode imputation for categoricals
    for c in categorical_cols:
        if c in result_df.columns:
            try:
                mode_row = (
                    result_df.groupBy(c)
                    .count()
                    .orderBy(F.desc("count"))
                    .first()
                )
                mode_val = (
                    mode_row[0]
                    if mode_row and mode_row[0] is not None
                    else categorical_mode
                )
            except Exception:
                mode_val = categorical_mode

            result_df = result_df.withColumn(
                c,
                F.when(F.col(c).isNull(), F.lit(mode_val)).otherwise(F.col(c)),
            )

    return result_df


def encode_categorical_features(df, categorical_cols, spark_context, categorical_mappings=None):
    """Encode categorical features to integer indices."""
    if categorical_mappings is None:
        categorical_mappings = {}

    result_df = df

    for c in categorical_cols:
        if c in categorical_mappings:
            mapping = categorical_mappings[c]
            b = spark_context.broadcast(mapping)
            result_df = result_df.withColumn(
                c + "_idx",
                F.udf(lambda s: int(b.value.get(str(s), 0)), IntegerType())(
                    F.coalesce(F.col(c).cast("string"), F.lit("UNKNOWN"))
                ),
            )
        else:
            vals = [r[0] for r in result_df.select(c).distinct().collect()]
            mapping = {v: i for i, v in enumerate(sorted([str(x) for x in vals]))}
            categorical_mappings[c] = mapping

            b = spark_context.broadcast(mapping)
            result_df = result_df.withColumn(
                c + "_idx",
                F.udf(lambda s: int(b.value.get(str(s), 0)), IntegerType())(
                    F.coalesce(F.col(c).cast("string"), F.lit("UNKNOWN"))
                ),
            )

    return result_df, categorical_mappings


def get_feature_columns():
    """Get the list of feature columns in the correct order."""
    return [
        "first_acct_val_amt",
        "first_face_amt",
        "first_cash_val_amt",
        "wc_total_assets",
        "wc_assetmix_stocks",
        "wc_assetmix_bonds",
        "wc_assetmix_mutual_funds",
        "wc_assetmix_annuity",
        "wc_assetmix_deposits",
        "wc_assetmix_other_assets",
        "psn_age",
        "stock_allocation_ratio",
        "bond_allocation_ratio",
        "annuity_allocation_ratio",
        "mutual_fund_allocation_ratio",
        "age_at_first_policy",
        "years_to_second_policy",
        "first_product_category_idx",
        "client_seg_idx",
        "client_seg_1_idx",
        "aum_band_idx",
        "channel_idx",
        "agent_segment_idx",
        "branchoffice_code_idx",
        "season_of_first_policy_idx",
    ]


print("✓ Preprocessing functions loaded")


In [ ]:
# ============================================================================
# MODEL AND ARTIFACTS LOADING
# ============================================================================

def load_model_and_artifacts() -> Dict[str, Any]:
    """Load the LightGBM model and its preprocessing artifacts."""
    client = MlflowClient()

    # Load registered LightGBM model
    model_uri = f"models:/{MODEL_NAME}/{MODEL_VERSION}"
    print(f"Loading model from {model_uri}...")
    lgbm_model = mlflow.lightgbm.load_model(model_uri)
    print("✓ Model loaded successfully")

    # Try to load artifacts from local path first
    artifacts = None
    if ARTIFACTS_LOCAL_PATH and os.path.exists(ARTIFACTS_LOCAL_PATH):
        print(f"Loading artifacts from local path: {ARTIFACTS_LOCAL_PATH}")
        with open(ARTIFACTS_LOCAL_PATH, "rb") as f:
            artifacts = pickle.load(f)
        print("✓ Artifacts loaded from local path")
    else:
        # Try MLflow run artifacts
        try:
            mv = client.get_model_version(name=MODEL_NAME, version=MODEL_VERSION)
            run_id = mv.run_id
            print(f"Attempting to download artifacts from run {run_id}...")
            artifacts_dir = mlflow.artifacts.download_artifacts(
                artifact_uri=f"runs:/{run_id}/artifacts"
            )
            artifacts_path = os.path.join(artifacts_dir, "artifacts.pkl")
            if os.path.exists(artifacts_path):
                with open(artifacts_path, "rb") as f:
                    artifacts = pickle.load(f)
                print("✓ Artifacts loaded from MLflow")
        except Exception as e:
            print(f"⚠ Could not load artifacts from MLflow: {e}")

    if artifacts is None:
        raise FileNotFoundError(
            f"Could not find artifacts.pkl. Please ensure it exists at {ARTIFACTS_LOCAL_PATH} "
            "or is available in MLflow."
        )

    prod2id = artifacts.get("prod2id")
    id2prod = artifacts.get("id2prod")
    label_map = artifacts.get("label_map")
    num_classes = artifacts.get("num_classes")
    categorical_mappings = artifacts.get("categorical_mappings")
    feature_cols = artifacts.get("feature_cols") or get_feature_columns()

    if prod2id is None or id2prod is None:
        raise ValueError("prod2id/id2prod mappings missing in artifacts.pkl")
    if label_map is None:
        raise ValueError("label_map missing in artifacts.pkl")

    print("✓ All artifacts loaded successfully")
    return {
        "model": lgbm_model,
        "prod2id": prod2id,
        "id2prod": id2prod,
        "label_map": label_map,
        "num_classes": num_classes,
        "categorical_mappings": categorical_mappings,
        "feature_cols": feature_cols,
    }


# Load model and artifacts
ctx = load_model_and_artifacts()
model = ctx["model"]
id2prod = ctx["id2prod"]
label_map = ctx["label_map"]
categorical_mappings = ctx["categorical_mappings"]
feature_cols = ctx["feature_cols"]

# Create inverse label map for predictions
inv_label_map = {v: k for k, v in label_map.items()}
final_id2prod = {
    model_id: id2prod[orig_id]
    for model_id, orig_id in inv_label_map.items()
    if orig_id in id2prod
}

print(f"✓ Model context loaded: {len(feature_cols)} features, {len(final_id2prod)} product classes")


In [ ]:
# ============================================================================
# LOAD PREDICTIONS FROM TABLE
# ============================================================================

print("\n" + "=" * 80)
print("LOADING PREDICTIONS FROM TABLE")
print("=" * 80)

# Load predictions table
df_predictions = spark.table(PREDICTIONS_TABLE)

# Optionally filter by business_month or other criteria
# df_predictions = df_predictions.filter(F.col("business_month") == "202512")

print(f"✓ Loaded {df_predictions.count():,} predictions from table")
print(f"\n  Columns in predictions table:")
for col in df_predictions.columns:
    print(f"    - {col}")

# Convert to pandas for easier manipulation (sample if too large)
MAX_SAMPLES = 10000  # Adjust based on your needs
pred_count = df_predictions.count()
if pred_count > MAX_SAMPLES:
    print(f"\n⚠ Large dataset ({pred_count:,} records). Sampling {MAX_SAMPLES:,} records for analysis...")
    df_predictions_sample = df_predictions.sample(fraction=MAX_SAMPLES / pred_count, seed=42)
    pred_pd = df_predictions_sample.toPandas()
else:
    pred_pd = df_predictions.toPandas()

print(f"✓ Working with {len(pred_pd):,} predictions")


In [ ]:
# ============================================================================
# RECONSTRUCT FEATURE DATA FOR SHAP ANALYSIS
# ============================================================================

print("\n" + "=" * 80)
print("RECONSTRUCTING FEATURE DATA")
print("=" * 80)

# Get cont_ids from predictions
cont_ids = pred_pd["cont_id"].unique().tolist()
print(f"✓ Found {len(cont_ids):,} unique cont_ids")

# Load source data for these cont_ids
df_raw = spark.table(SOURCE_TABLE)
df_raw = df_raw.filter(F.col("cont_id").isin(cont_ids))
df_raw = df_raw.filter(F.col("policy_status") == "Active")

# Create product_category
df_raw = create_product_category_column(df_raw)

# Filter to single-policy clients (matching production logic)
df_events = df_raw.select(
    "cont_id",
    "axa_party_id",
    "product_category",
    "register_date",
    "isrd_brth_date",
    "acct_val_amt",
    "face_amt",
    "cash_val_amt",
    "wc_total_assets",
    "wc_assetmix_stocks",
    "wc_assetmix_bonds",
    "wc_assetmix_mutual_funds",
    "wc_assetmix_annuity",
    "wc_assetmix_deposits",
    "wc_assetmix_other_assets",
    "psn_age",
    "client_seg",
    "client_seg_1",
    "aum_band",
    "channel",
    "agent_segment",
    "branchoffice_code",
    "policy_no",
).filter(
    (F.col("axa_party_id").isNotNull())
    & (F.col("cont_id").isNotNull())
    & (F.col("register_date").isNotNull())
    & (F.col("product_category").isNotNull())
)

# Count policies per axa_party_id
party_counts = (
    df_events.groupBy("axa_party_id")
    .agg(F.countDistinct("policy_no").alias("policy_count"))
    .filter(F.col("policy_count") == 1)  # Exactly 1 policy
)

# Filter to single-policy clients
df_single_policy = df_events.join(
    party_counts.select("axa_party_id"), on="axa_party_id", how="inner"
).dropDuplicates(["cont_id"])  # Remove duplicates

print(f"✓ Filtered to {df_single_policy.count():,} single-policy client records")

# Convert dates
df_single_policy = df_single_policy.withColumn("register_ts", F.to_timestamp("register_date"))
df_single_policy = df_single_policy.withColumn("birth_ts", F.to_timestamp("isrd_brth_date"))

# Create "first policy" features
df_first = df_single_policy.select(
    "cont_id",
    "axa_party_id",
    F.col("product_category").alias("first_product_category"),
    F.col("register_ts").alias("first_register_ts"),
    "birth_ts",
    F.col("acct_val_amt").alias("first_acct_val_amt"),
    F.col("face_amt").alias("first_face_amt"),
    F.col("cash_val_amt").alias("first_cash_val_amt"),
    "wc_total_assets",
    "wc_assetmix_stocks",
    "wc_assetmix_bonds",
    "wc_assetmix_mutual_funds",
    "wc_assetmix_annuity",
    "wc_assetmix_deposits",
    "wc_assetmix_other_assets",
    "psn_age",
    "client_seg",
    "client_seg_1",
    "aum_band",
    "channel",
    "agent_segment",
    "branchoffice_code",
)

# Add temporal features
df_first = df_first.withColumn("second_register_ts", F.current_timestamp())
df_first = add_asset_allocation_ratios(df_first)

df_first = df_first.withColumn(
    "season_of_first_policy",
    F.when(F.month("first_register_ts").between(1, 3), "Q1")
    .when(F.month("first_register_ts").between(4, 6), "Q2")
    .when(F.month("first_register_ts").between(7, 9), "Q3")
    .when(F.month("first_register_ts").between(10, 12), "Q4")
    .otherwise("Unknown"),
)

df_first = df_first.withColumn(
    "age_at_first_policy",
    F.datediff(F.col("first_register_ts"), F.col("birth_ts")) / 365.25,
)

df_first = df_first.withColumn(
    "years_to_second_policy",
    F.datediff(F.col("second_register_ts"), F.col("first_register_ts")) / 365.25,
)

# Impute missing values
categorical_cols = [
    "first_product_category",
    "client_seg",
    "client_seg_1",
    "aum_band",
    "channel",
    "agent_segment",
    "branchoffice_code",
    "season_of_first_policy",
]

df_first = impute_missing_values(df_first, categorical_cols)

# Encode categorical features
df_encoded, _ = encode_categorical_features(
    df_first, categorical_cols, spark.sparkContext, categorical_mappings=categorical_mappings
)

# Convert to pandas
select_cols = ["cont_id", "axa_party_id"] + feature_cols
features_pd = df_encoded.select(select_cols).toPandas()
features_pd.fillna(0, inplace=True)

print(f"✓ Reconstructed features for {len(features_pd):,} clients")
print(f"✓ Feature columns: {len(feature_cols)}")


In [ ]:
# ============================================================================
# MERGE PREDICTIONS WITH FEATURES
# ============================================================================

# Merge predictions with features
pred_with_features = pred_pd.merge(
    features_pd[["cont_id"] + feature_cols], on="cont_id", how="inner"
)

print(f"✓ Merged predictions with features: {len(pred_with_features):,} records")

# Ensure we have feature data for all predictions
if len(pred_with_features) < len(pred_pd):
    missing = len(pred_pd) - len(pred_with_features)
    print(f"⚠ Warning: {missing} predictions missing feature data (will be skipped)")


In [ ]:
# ============================================================================
# SHAP ANALYSIS FOR REASONING
# ============================================================================

print("\n" + "=" * 80)
print("COMPUTING SHAP VALUES FOR REASONING")
print("=" * 80)

# Prepare feature matrix
X = pred_with_features[feature_cols].values

print(f"✓ Feature matrix shape: {X.shape}")

# Compute SHAP values
print("Computing SHAP values (this may take a while)...")
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Get predictions to determine which class SHAP values to use
pred_probs = model.predict(X)
pred_class_ids = np.argmax(pred_probs, axis=1)

# Handle SHAP output format
if isinstance(shap_values, list):
    # List of arrays: one array per class
    shap_array = np.array([shap_values[pred_class_ids[i]][i] for i in range(len(pred_with_features))])
elif isinstance(shap_values, np.ndarray) and len(shap_values.shape) == 3:
    # 3D array: (n_samples, n_features, n_classes) or (n_classes, n_samples, n_features)
    n_samples, n_features, n_classes = shap_values.shape
    if n_samples == len(pred_with_features) and n_features == len(feature_cols):
        shap_array = np.array([shap_values[i, :, pred_class_ids[i]] for i in range(n_samples)])
    else:
        shap_values = shap_values.transpose(1, 0, 2)  # Try transpose
        shap_array = np.array([shap_values[i, :, pred_class_ids[i]] for i in range(len(pred_with_features))])
else:
    shap_array = shap_values

print(f"✓ Computed SHAP values: {shap_array.shape}")

# Also compute SHAP values for all classes (for comparative analysis)
if isinstance(shap_values, list):
    shap_all_classes = shap_values  # Already in list format
elif isinstance(shap_values, np.ndarray) and len(shap_values.shape) == 3:
    # Convert to list format for easier access
    if shap_values.shape[0] == len(pred_with_features):
        shap_all_classes = [shap_values[:, :, i] for i in range(shap_values.shape[2])]
    else:
        shap_all_classes = [shap_values[i, :, :].T for i in range(shap_values.shape[0])]
else:
    shap_all_classes = [shap_array]  # Single class

print(f"✓ SHAP analysis complete")


In [ ]:
# ============================================================================
# GENERATE DETAILED REASONING FOR EACH PREDICTION
# ============================================================================

print("\n" + "=" * 80)
print("GENERATING DETAILED REASONING")
print("=" * 80)


def format_feature_value(feat_name: str, feat_value: Any, categorical_mappings: Dict) -> str:
    """Format feature value for human-readable display."""
    if feat_name.endswith("_idx"):
        base_name = feat_name.replace("_idx", "")
        if base_name in categorical_mappings:
            mapping = categorical_mappings[base_name]
            inv_mapping = {v: k for k, v in mapping.items()}
            try:
                idx_val = int(float(feat_value))
                return inv_mapping.get(idx_val, str(feat_value))
            except:
                return str(feat_value)
    
    # Format numeric values
    if isinstance(feat_value, (int, float)):
        if "ratio" in feat_name or "allocation" in feat_name:
            return f"{feat_value:.1%}"
        elif "age" in feat_name or "years" in feat_name:
            return f"{feat_value:.1f}"
        elif "amt" in feat_name or "assets" in feat_name or "mix" in feat_name:
            return f"${feat_value:,.0f}"
        else:
            return f"{feat_value:,.2f}"
    
    return str(feat_value)


def generate_reasoning(
    cont_id: str,
    pred_product: str,
    pred_prob: float,
    all_probs: np.ndarray,
    feature_values: pd.Series,
    shap_contributions: np.ndarray,
    shap_all_classes: List[np.ndarray],
    feature_cols: List[str],
    final_id2prod: Dict[int, str],
    categorical_mappings: Dict,
) -> Dict[str, Any]:
    """Generate detailed reasoning for a single prediction."""
    
    # Get top contributing features (positive and negative)
    feature_contributions = list(zip(feature_cols, shap_contributions))
    feature_contributions.sort(key=lambda x: abs(float(x[1])), reverse=True)
    
    top_positive = [fc for fc in feature_contributions if fc[1] > 0][:5]
    top_negative = [fc for fc in feature_contributions if fc[1] < 0][:5]
    
    # Get top 3 alternative products (by probability)
    prob_with_class = [(all_probs[i], i) for i in range(len(all_probs))]
    prob_with_class.sort(reverse=True)
    
    top_alternatives = []
    for prob, class_id in prob_with_class[1:4]:  # Skip the predicted one
        product_name = final_id2prod.get(class_id, f"CLASS_{class_id}")
        top_alternatives.append({
            "product": product_name,
            "probability": float(prob),
            "class_id": int(class_id),
        })
    
    # Get SHAP contributions for top alternative
    alternative_reasons = []
    if top_alternatives and len(shap_all_classes) > 1:
        alt_class_id = top_alternatives[0]["class_id"]
        if alt_class_id < len(shap_all_classes):
            # Get SHAP values for alternative class
            if isinstance(shap_all_classes[alt_class_id], np.ndarray):
                if len(shap_all_classes[alt_class_id].shape) == 2:
                    alt_shap = shap_all_classes[alt_class_id][0]  # First sample
                else:
                    alt_shap = shap_all_classes[alt_class_id]
            else:
                alt_shap = shap_all_classes[alt_class_id]
            
            # Features that favor alternative over predicted
            diff_contributions = [
                (feature_cols[i], alt_shap[i] - shap_contributions[i]) 
                for i in range(len(feature_cols))
            ]
            diff_contributions.sort(key=lambda x: abs(float(x[1])), reverse=True)
            
            alternative_reasons = [
                {
                    "feature": fc[0],
                    "value": format_feature_value(fc[0], feature_values[fc[0]], categorical_mappings),
                    "contribution_diff": float(fc[1]),
                }
                for fc in diff_contributions[:3]
            ]
    
    # Build reasoning text
    reasoning_parts = []
    
    # Why this product was recommended
    reasoning_parts.append("## Why This Product Was Recommended")
    reasoning_parts.append(f"The model predicts **{pred_product}** with {pred_prob:.1%} confidence.")
    reasoning_parts.append("\n**Top Contributing Features (Supporting Prediction):**")
    
    for i, (feat_name, shap_val) in enumerate(top_positive[:5], 1):
        feat_value = format_feature_value(feat_name, feature_values[feat_name], categorical_mappings)
        impact = "🔥 High" if abs(shap_val) > 0.1 else "⭐ Moderate" if abs(shap_val) > 0.05 else "📊 Low"
        reasoning_parts.append(f"{i}. **{feat_name}** = {feat_value} ({impact} impact: +{shap_val:.4f})")
    
    if top_negative:
        reasoning_parts.append("\n**Features Reducing Confidence:**")
        for i, (feat_name, shap_val) in enumerate(top_negative[:3], 1):
            feat_value = format_feature_value(feat_name, feature_values[feat_name], categorical_mappings)
            reasoning_parts.append(f"{i}. **{feat_name}** = {feat_value} (reducing confidence: {shap_val:.4f})")
    
    # Confidence score explanation
    reasoning_parts.append("\n## Confidence Score Analysis")
    if pred_prob >= 0.7:
        reasoning_parts.append(f"**High Confidence ({pred_prob:.1%})**: Strong feature alignment with {pred_product} patterns.")
    elif pred_prob >= 0.5:
        reasoning_parts.append(f"**Moderate Confidence ({pred_prob:.1%})**: Reasonable alignment, but some uncertainty remains.")
    else:
        reasoning_parts.append(f"**Low Confidence ({pred_prob:.1%})**: Weak feature alignment; consider reviewing alternative products.")
    
    # Why other products weren't predicted
    reasoning_parts.append("\n## Why Other Products Were Not Predicted")
    reasoning_parts.append("**Top Alternative Products:**")
    
    for alt in top_alternatives:
        prob_diff = pred_prob - alt["probability"]
        reasoning_parts.append(f"- **{alt['product']}**: {alt['probability']:.1%} probability (gap: {prob_diff:.1%})")
    
    if alternative_reasons:
        reasoning_parts.append("\n**Key Differences from Top Alternative:**")
        for reason in alternative_reasons[:3]:
            reasoning_parts.append(f"- **{reason['feature']}** = {reason['value']} (contribution difference: {reason['contribution_diff']:+.4f})")
    
    return {
        "cont_id": cont_id,
        "predicted_product": pred_product,
        "prediction_confidence": pred_prob,
        "reasoning_text": "\n".join(reasoning_parts),
        "top_positive_features": [
            {
                "feature": feat_name,
                "value": format_feature_value(feat_name, feature_values[feat_name], categorical_mappings),
                "shap_contribution": float(shap_val),
            }
            for feat_name, shap_val in top_positive[:5]
        ],
        "top_negative_features": [
            {
                "feature": feat_name,
                "value": format_feature_value(feat_name, feature_values[feat_name], categorical_mappings),
                "shap_contribution": float(shap_val),
            }
            for feat_name, shap_val in top_negative[:3]
        ],
        "top_alternatives": top_alternatives,
        "alternative_reasons": alternative_reasons,
    }


# Generate reasoning for all predictions
reasoning_list = []
print(f"Generating reasoning for {len(pred_with_features):,} predictions...")

for idx in range(len(pred_with_features)):
    if (idx + 1) % 1000 == 0:
        print(f"  Processed {idx + 1:,} / {len(pred_with_features):,}...")
    
    row = pred_with_features.iloc[idx]
    
    # Get SHAP values for this prediction
    if isinstance(shap_all_classes, list) and len(shap_all_classes) > 0:
        pred_class_id = int(row["pred_class_id"]) if "pred_class_id" in row else pred_class_ids[idx]
        if pred_class_id < len(shap_all_classes):
            if isinstance(shap_all_classes[pred_class_id], np.ndarray):
                if len(shap_all_classes[pred_class_id].shape) == 2:
                    shap_for_pred = shap_all_classes[pred_class_id][idx]
                else:
                    shap_for_pred = shap_all_classes[pred_class_id]
            else:
                shap_for_pred = shap_array[idx]
        else:
            shap_for_pred = shap_array[idx]
    else:
        shap_for_pred = shap_array[idx]
    
    # Get all probabilities
    all_probs = pred_probs[idx]
    
    reasoning = generate_reasoning(
        cont_id=str(row["cont_id"]),
        pred_product=str(row.get("predicted_product", row.get("pred_product", "UNKNOWN"))),
        pred_prob=float(row["pred_prob"]),
        all_probs=all_probs,
        feature_values=row[feature_cols],
        shap_contributions=shap_for_pred,
        shap_all_classes=shap_all_classes,
        feature_cols=feature_cols,
        final_id2prod=final_id2prod,
        categorical_mappings=categorical_mappings,
    )
    
    reasoning_list.append(reasoning)

print(f"✓ Generated reasoning for {len(reasoning_list):,} predictions")


In [ ]:
# ============================================================================
# CREATE REASONING DATAFRAME AND SAVE
# ============================================================================

print("\n" + "=" * 80)
print("CREATING REASONING OUTPUT")
print("=" * 80)

# Create DataFrame with reasoning
reasoning_df = pd.DataFrame(reasoning_list)

# Merge back with original predictions
final_reasoning_df = pred_pd.merge(
    reasoning_df[["cont_id", "reasoning_text", "top_positive_features", "top_negative_features", "top_alternatives", "alternative_reasons"]],
    on="cont_id",
    how="left"
)

print(f"✓ Created reasoning DataFrame: {len(final_reasoning_df):,} records")
print(f"\n  Columns:")
for col in final_reasoning_df.columns:
    print(f"    - {col}")

# Display sample reasoning
print("\n" + "=" * 80)
print("SAMPLE REASONING OUTPUT")
print("=" * 80)

sample_idx = 0
if len(final_reasoning_df) > 0:
    sample = final_reasoning_df.iloc[sample_idx]
    print(f"\nClient: {sample['cont_id']}")
    print(f"Predicted Product: {sample.get('predicted_product', sample.get('pred_product', 'N/A'))}")
    print(f"Confidence: {sample['pred_prob']:.1%}")
    print(f"\nReasoning:\n{sample['reasoning_text']}")

# Convert to Spark DataFrame
final_reasoning_spark = spark.createDataFrame(final_reasoning_df)

# # Optionally save to table
# REASONING_TABLE = "eda_smartlist.us_wealth_management_smartlist.ML_predictions_reasoning"

# try:
#     final_reasoning_spark.write.mode("overwrite").saveAsTable(REASONING_TABLE)
#     print(f"\n✓ Reasoning saved to table: {REASONING_TABLE}")
# except Exception as e:
#     print(f"\n⚠ Could not save to table: {e}")
#     print("Reasoning DataFrame is available as 'final_reasoning_df' (pandas) and 'final_reasoning_spark' (Spark)")

print("\n" + "=" * 80)
print("REASONING ANALYSIS COMPLETE")
print("=" * 80)
